1)  
**a)** какая задача будет решаться (классификация/регрессия/кластеризация/еще что-то)

Здесь в принципе можно решать любую задачу в зависимости от данных, которые были найдены. Я нашла данные, состоящие из текста отзыва и оценки от 1 до 5. Поэтому в данном случае можно решать, например, задачу регрессии, что в принципе имеет смысл так как на метках есть порядок (5 > 1 и не обязательно считать эти метки категориальными). Также можно решать задачу класссификации на 5 классов. Я решала задачу классификации только на 2 класса, преобразовав исходные 5 меток в 0 или 1, так как в задании требовалось именно определение тональности вида positive / negative.

**b)** какими будут целевые значения (например, классы в задаче классификации) - почему именно такими, как прогнозы будут показываться в демонстрации (например: каким-то числом, текстом, цветом)

Как я сказала в предыдущем пункте я рассматриваю в качестве целевых значений 0 или 1 : негативный или позитивный отзыв соответственно. Выбрала именно такие метки так как во-первых, заданием было научиться отличать позитивные отзывы от негативных и во-вторых, эта задача чуть проще, чем классфикация на 5 классов и простым методам будет проще с ней справиться. В демонстрации я просто буду говорить какого вида данный отзыв : positive или negative.

**c)** как измерять качество, чтобы было более-менее интуитивно понятно, высокое оно или не очень

В силу того, что решается задача классификации на 2 класса, то у нас есть море метрик, которые мы можем рассматривать : accuracy, precision, recall, F1. Однако можно заметить, что классы у нас очень несбалансированы, негативных отзывов в разы больше. Поэтому accuracy лучше не брать. Лучше посчитать F1-меру.

**d)** на каких отзывах должна работать демонстрация (язык, длина, наличие ошибок и сленга в тексте).

Я парсила сайт отзывов на русские банки с отзывами на русском. Поэтому язык - русский. Длина должна быть не очень большой, так как я рассматривала не полный текст отзыва (далее объяснено почему), а лишь первые 3-6 предложений. К ошибкам модель скорее неустойчива так в качестве признакового описания отзыва используется bag of words. И хоть я и произвожу лемматизацию слов, одно и то же слово с ошибкой и без скорее всего распознается как разные слова. Чтобы избежать этого можно было бы использовать мешок из  n-gramm и тогда модель была бы менее чувствительна к ошибкам. К небольшому сленгу модель должна быть устойчива так как выборка содержит некоторые примеры со сленгом, хотя их скорее всего не очень много.



In [4]:
from bs4 import BeautifulSoup
import re
import numpy as np
import time
import urllib.request


small_responses = []
marks = []

Парсим страницы с отзывами с сайта banki.ru. На странице www.banki.ru/services/responses/list содержится много отзывов(по 25 на страницу). К почти каждому отзыву человек прилагает оценку от 1 до 5.

Однако эти отзывы ограничены в кол-ве символов и чтобы прочитать каждый отзыв полностью нужно переходить на другую страницу. Я не стала парсить каждый отзыв полностью так как для того, чтобы не забанили на данном сайте нужно ждать около 10-15 секунд между запросами и тогда парсинг полных отзывов занял бы чуть больше суток. 

In [17]:
for page in np.arange(140, 380):
    fp = urllib.request.urlopen("http://www.banki.ru/services/responses/list/?page=" + str(page))
    html = fp.read()
    html_doc = html.decode("utf8")
    fp.close()

    soup = BeautifulSoup(html_doc, 'html.parser')

    articles = soup.find_all('article')
    for article in articles:

        if 'Без оценки' in article.find_all('div')[1].get_text().strip():
            continue

        mark = re.findall(r"\d", article.find_all('div')[1].get_text())[0]
   
        title = article.find_all('div')[0].get_text().strip()
        small_response = article.find_all('div')[3].get_text()
        small_response = small_response.replace('Весь отзыв', '').strip()

        small_responses.append(title + ' ' + small_response)
        marks.append(mark)

    rand_sec = np.random.randint(7, 15)
    time.sleep(rand_sec)



In [21]:
len(small_responses) 

6972

Посмотрим на соотношение классов. Видим, что больше всего недовольных. Также видим, что в нашей выборке мало 3 и 4. 

In [26]:
marks = np.array(marks).astype(int)
np.unique(marks, return_counts=True)

(array([1, 2, 3, 4, 5]), array([3860,  691,  279,  214, 1928]))

Наша задача состоит не в предсказывании шкалы от 1 до 5, нам лишь нужно сказать плохой отзыв или нет. Поэтому объединим 1 и 2 в плохой отзыв, а 4 и 5 в хороший. С тройкой не понятно, что делать - кажется, что она бывает и плохим, и хорошим отзывом. Лучше не будем ее трогать, их все равно не много. 

In [27]:
indeces_considered = np.where(marks!=3)[0]

new_marks = marks[indeces_considered]

new_marks[new_marks==1] = 0
new_marks[new_marks==2] = 0
new_marks[new_marks==5] = 1
new_marks[new_marks==4] = 1


small_responses = np.array(small_responses)
new_responses = small_responses[indeces_considered]


In [28]:
import pymorphy2
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.svm import LinearSVC
from sklearn.linear_model import LogisticRegression
from joblib import load, dump
import pickle

Разбиваем данные на test и train

In [29]:
from sklearn.model_selection import train_test_split
train_responses, test_responses, train_marks, test_marks = train_test_split(new_responses, new_marks, test_size=0.1)

Делаем лемматизацию. Здесь можно было бы еще и стоп-слова убрать, но без них работает чуть похуже - наверно из-за небольшого размера выборки.

In [30]:
morph = pymorphy2.MorphAnalyzer()
for i, response in enumerate(train_responses):
    train_responses[i] = ' '.join([morph.parse(word)[0].normal_form for word in response.split()])
    
for i, response in enumerate(test_responses):
    test_responses[i] = ' '.join([morph.parse(word)[0].normal_form for word in response.split()])

В качестве признакового описания берем bag of words.

In [31]:
vectorizer = CountVectorizer()
train_bag_words = vectorizer.fit_transform(train_responses)
test_bag_words = vectorizer.transform(test_responses)

Будем обучать SVM с линейным ядром.

In [37]:
clf = LinearSVC()
clf.fit(train_bag_words, train_marks)
predictions = clf.predict(test_bag_words)

Смотрим на разные метрики. Видим, что в принципе для простой модели, небольшого размера выборки и текстовых данных получилось довольно неплохо. 

In [38]:
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score
print(accuracy_score(predictions, test_marks),
      f1_score(predictions, test_marks), 
      precision_score(predictions, test_marks), 
      recall_score(predictions, test_marks) )

0.8492537313432836 0.7455919395465994 0.7551020408163265 0.736318407960199


Посмотрим как будет работать логистическая регрессия. Видим, что мы получили результат сравнивнимо лучше по F1. Также recall и accuracy сильно выросли.

In [41]:
clf = LogisticRegression()
clf.fit(train_bag_words, train_marks)
predictions = clf.predict(test_bag_words)
print(accuracy_score(predictions, test_marks),
      f1_score(predictions, test_marks), 
      precision_score(predictions, test_marks), 
      recall_score(predictions, test_marks) )

0.8776119402985074 0.7807486631016043 0.7498979591836735 0.8202247191011236


Сохраняем модель и vectorizer.

In [36]:
with open('model_dump_log.pkl', 'wb') as output_file:
    pickle.dump(clf, output_file)
    
with open('vectorizer_dump.pkl', 'wb') as output_file:
    pickle.dump(vectorizer, output_file)